In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%pwd

'/Users/IvanTang/quant/IMC_Prosperity3_GraniteFlow/src/analysis/round2'

In [ ]:
df1 = pd.read_csv('../../../data/round-2-island-data-bottle/prices_round_2_day_-1.csv', delimiter=';')
df2 = pd.read_csv('../../../data/round-2-island-data-bottle/prices_round_2_day_0.csv', delimiter=';')
df3 = pd.read_csv('../../../data/round-2-island-data-bottle/prices_round_2_day_1.csv', delimiter=';')
df = pd.concat([df1, df2, df3])

In [8]:
print(df)

      day;timestamp;product;bid_price_1;bid_volume_1;bid_price_2;bid_volume_2;bid_price_3;bid_volume_3;ask_price_1;ask_volume_1;ask_price_2;ask_volume_2;ask_price_3;ask_volume_3;mid_price;profit_and_loss
0      -1;0;CROISSANTS;4304;112;;;;;4305;112;;;;;4304...                                                                                                                                                   
1      -1;0;JAMS;6670;66;6669;137;;;6671;66;6672;137;...                                                                                                                                                   
2      -1;0;SQUID_INK;2005;1;2002;31;;;2006;31;;;;;20...                                                                                                                                                   
3      -1;0;PICNIC_BASKET1;59284;20;59283;18;;;59294;...                                                                                                                                